In [1]:
import gradio as gr

In [2]:
import fitz 

In [3]:
import google.generativeai as genai

In [4]:
genai.configure(api_key='xxxxxxxx')

In [5]:
def extract_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text= ""
    for pages in doc:
        full_text += pages.get_text()
    return full_text

In [6]:
def ask_question(question, combined_text):
    model = genai.GenerativeModel("models/gemini-2.0-flash")
    prompt= f"""
    You are an expert analyst . Use the following combined pdf data to answer the questions of the user . Give in the clear format.
    PDF Data : 
    \"\"\"{combined_text}\"\"\"
    User Question : {question}
    Answer :
    """
    response = model.generate_content(prompt)
    return response.text.strip()

In [7]:
def process (pdf1, pdf2 , question):
    if pdf1 is None or pdf2 is None:
        return "Please Provide PDF "
    if not question.strip():
        return "Please ask me question"
    text1 = extract_from_pdf(pdf1.name)
    text2 = extract_from_pdf(pdf2.name)
    combined_text = text1 + "\n\n"  + text2
    answer = ask_question(question, combined_text)
    return answer

In [8]:
with gr.Blocks() as demo:
    gr.Markdown("*** Welcome to The AI Agent for Analysing PDF ***")
    pdf1 = gr.File(label="Upload PDF 1", file_types=['.pdf'],interactive=True)
    pdf2 = gr.File(label="Upload PDF 2", file_types=['.pdf'],interactive=True)
    question = gr.Textbox(label="Ask Your quetion",lines=2)
    answer = gr.Textbox(label="Answer from Gemini",lines=10)
    btn = gr.Button("Get Answer from your AI ")
    btn.click(process, inputs=[pdf1, pdf2, question],outputs=answer)
    

In [9]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
